- http://mlexplained.com/2018/02/15/language-modeling-tutorial-in-torchtext-practical-torchtext-part-2/
- http://anie.me/On-Torchtext/
- https://github.com/mjc92/TorchTextTutorial/blob/master/01.%20Getting%20started.ipynb
- https://colab.research.google.com/drive/1bcsRgkCpiFxgvmMMAipr-itLIsl4QQCz#scrollTo=2cOQ5YwCvY58
- https://medium.com/dair-ai/building-rnns-is-fun-with-pytorch-and-google-colab-3903ea9a3a79

In [1]:
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [0]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm

In [0]:
import shutil
from pathlib import Path

In [0]:
import spacy
import torch
import torch.nn as nn
import torchtext as tt

In [5]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
PATH = Path('/gdrive/My Drive/data/flick8k')
images = PATH/'Flicker8k_Dataset'
images_train = images/'train'
images_test = images/'test'

## Data

In [0]:
spacy_en = spacy.load('en')

In [0]:
def tokenizer(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [0]:
# embedding dimension (size of the word vectors)
embed_size = 300
# batch size
batch_size = 64
# BPTT (backpropagation through time) length
seq_len = 16

### Flickr8k

In [177]:
df = pd.read_csv(PATH/'Flickr8k.token.txt', names=['id', 'caption'], delimiter='\t') 
df['number'] = df['id'].apply(lambda x: int(x.split('#')[1]))
df['id'] = df['id'].apply(lambda x: x.split('#')[0])
df.head()

,id,caption,number
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...,0
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .,1
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .,2
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...,3
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...,4


In [0]:
size = len(df)
train_size = int(size * 0.9)
valid_size = size - train_size

#### Save text data

In [0]:
df[:train_size].to_csv(PATH/'train.csv',columns=['caption'], header=False, index=False)
df[train_size:].to_csv(PATH/'valid.csv',columns=['caption'], header=False, index=False)

#### Load text data

In [0]:
TEXT = tt.data.Field(tokenize=tokenizer,
                     lower=True,
                     batch_first=True,
                     init_token='<bos>',
                     eos_token='<eos>',
                     fix_length=seq_len)

In [0]:
train, valid = tt.data.TabularDataset.splits(path=PATH,
                                             train='train.csv',
                                             validation='valid.csv',
                                             format='csv',
                                             fields=[('text', TEXT)])

In [182]:
# print information about the data
print('train.fields', train.fields)
print('len(train)', len(train))
print('vars(train[0])', vars(train[0])['text'][:10])

train.fields {'text': <torchtext.data.field.Field object at 0x7fe22427f128>}
len(train) 36414
vars(train[0]) ['a', 'child', 'in', 'a', 'pink', 'dress', 'is', 'climbing', 'up', 'a']


In [183]:
print(TEXT.pad([vars(train[0])['text']]))

[['<bos>', 'a', 'child', 'in', 'a', 'pink', 'dress', 'is', 'climbing', 'up', 'a', 'set', 'of', 'stairs', 'in', '<eos>']]


In [185]:
len(train), len(valid)

(36414, 4046)

#### Load Vocab
Check list of vocab vectors [here](https://torchtext.readthedocs.io/en/latest/vocab.html#torchtext.vocab.Vocab.load_vectors).

In [0]:
TEXT.build_vocab(train, vectors='glove.6B.300d')

In [187]:
vocab = TEXT.vocab
print('Vocabulary shape:', vocab.vectors.shape)

Vocabulary shape: torch.Size([8161, 300])


#### Iterator
transform the train/valid dataset to make them compatible with [bpttiterator](https://torchtext.readthedocs.io/en/latest/data.html#bpttiterator).

In [0]:
def dataset2example(dataset, field):
    examples = list(map(lambda example: ['_bos_']+ example.text + ['_eos_'], dataset.examples))
    examples = [item for example in examples for item in example]
    example = tt.data.Example()
    setattr(example, 'text', examples)
    return tt.data.Dataset([example], fields={'text': field})

In [0]:
train_example = dataset2example(train, TEXT)
valid_example = dataset2example(valid, TEXT)

In [0]:
train_iter, valid_iter = tt.data.BPTTIterator.splits((train_example, valid_example), batch_size=batch_size, bptt_len=30)

In [191]:
len(train_iter), len(valid_iter)

(262, 30)

look at data

In [192]:
example = train_iter.dataset[0]
print(example.text) # tokens of the sample at index 0

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [193]:
batch = next(iter(train_iter))
vars(batch).keys()

dict_keys(['batch_size', 'dataset', 'fields', 'text', 'target'])

In [194]:
batch = next(iter(valid_iter))
vars(batch).keys()

dict_keys(['batch_size', 'dataset', 'fields', 'text', 'target'])

### Model

In [0]:
class SimpleLM(nn.Module):
    def __init__(self, vocab, embed_dim, batch_size, num_layers=1, bidirectional=False, hidden_state=None):
        super().__init__()
        self.vocab_size = len(vocab)
        self.embed_dim, self.batch_size, self.num_layers = embed_dim, batch_size, num_layers
        self.num_directions = 2 if bidirectional==True else 1
        # emebedding layer
        self.embed = nn.Embedding(self.vocab_size, embed_dim)
        self.embed.weight.data.copy_(vocab.vectors)
        # lstm layer
        self.init_hidden(hidden_state)
        self.lstm = nn.LSTM(embed_dim, embed_dim, num_layers, bidirectional=bidirectional)
        self.batchnorm = nn.BatchNorm1d(batch_size)
        # output layer
        self.dense = nn.Linear(embed_dim, self.vocab_size)
        
    def init_hidden(self, hidden_state=None):
        if hidden_state is not None:
            self.hidden_state = hidden_state
            return
        # num_layers * num_directions, batch, hidden_size
        dim1 = self.num_layers * self.num_directions
        h = torch.zeros(dim1, self.batch_size, self.embed_dim).cuda()
        c = torch.zeros(dim1, self.batch_size, self.embed_dim).cuda()
        self.hidden_state = (h, c)
        
    def forward(self, X, lengths=None):
        embed_out = self.embed(X) # (bs, embed_dim)
        lstm_out, hidden_state = self.lstm(embed_out, self.hidden_state)
        self.hidden_state = hidden_state #.detach()
        batchnorm_out = self.batchnorm(lstm_out)
        output = self.dense(batchnorm_out)
        return output

In [0]:
model = SimpleLM(TEXT.vocab, embed_size, batch_size).cuda()

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=(0.7, 0.99))

In [198]:
n_tokens = vocab.vectors.size(0); n_tokens

8161

### Train

In [0]:
epochs = 10

In [0]:
def run_train_epoch(epoch):
    """One epoch of a training loop"""
    # train mode
    model.train()
    train_loss = 0
    for batch in tqdm(train_iter):
        # reset the hidden state
        model.init_hidden()
        text, target = batch.text.cuda(), batch.target.cuda()
        prediction = model(text)
        loss = criterion(prediction.view(-1, n_tokens), target.view(-1))
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        train_loss += loss.item()
        
    train_loss /= len(train_iter)

    # eval mode & deactivate autograd engine 
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in valid_iter:
            model.init_hidden()
            text, target = batch.text.cuda(), batch.target.cuda()
            prediction = model(text)
            loss = criterion(prediction.view(-1, n_tokens), target.view(-1))
            val_loss += loss.item()
    val_loss /= len(valid_iter)
    # record epoch stats
    print('Epoch: {}, Training Loss: {:.4f}, Validation Loss: {:.4f}'.format(epoch, train_loss, val_loss))


In [201]:
epochs = 2
for epoch in range(1, epochs + 1):
    run_train_epoch(epoch)


  0%|          | 1/262 [00:00<00:30,  8.56it/s]

Epoch: 1, Training Loss: 4.0877, Validation Loss: 3.1586



100%|██████████| 262/262 [00:04<00:00, 56.75it/s]

Epoch: 2, Training Loss: 2.9167, Validation Loss: 2.9422


### Inference

In [202]:
model.eval()

SimpleLM(
  (embed): Embedding(8161, 300)
  (lstm): LSTM(300, 300)
  (batchnorm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense): Linear(in_features=300, out_features=8161, bias=True)
)

In [0]:
i2w = TEXT.vocab.itos
w2i = dict([(w,i) for i,w in enumerate(TEXT.vocab.itos)])

In [0]:
def pad_text(text, target_length, padding=TEXT.pad_token):
    current_length = len(text)
    for i in range(current_length, target_length):
        text.append(padding)
    return text

In [0]:
text = pad_text([TEXT.init_token, 'a', 'boy'], batch_size, TEXT.init_token)
in_tensor = torch.tensor([w2i[w] for w in text])
in_tensor = in_tensor.cuda()

In [0]:
in_tensor = in_tensor.view(1, batch_size)
out_tensor = model(in_tensor)

In [268]:
out_array = out_tensor.cpu().data.numpy()
indexes = np.argmax(out_array, axis=2)

' '.join([i2w[i] for i in indexes.tolist()[0]])

'. man in a while a a . a a . a . a two a a . two a . . . . a a a a a a a a a a a a a two a a a a a a a a a . a a a a with a a a a a a a two a a a'